**⭐ 1. What This Pattern Solves**

Efficiently tracks the most recently used (MRU) or least recently used (LRU) items in a dataset or cache.

Useful in caching pipelines, ETL intermediate storage, or session/state management.

Ensures that old/unneeded entries are evicted automatically when memory limits are reached.

Helps avoid expensive recomputation or repeated I/O for frequently accessed data.

**⭐ 2. SQL Equivalent**

In [0]:
%sql
-- Track most recent access timestamp
SELECT key, value
FROM cache_table
ORDER BY last_accessed DESC
LIMIT 5;

**⭐ 3. Core Idea**

Maintain insertion order while updating access order; remove oldest items when capacity exceeds limit.

**⭐ 4. Template Code (MEMORIZE THIS)**

In [0]:
from collections import OrderedDict

def lru_cache(capacity):
    cache = OrderedDict()
    
    def get(key):
        if key not in cache:
            return -1
        cache.move_to_end(key)  # mark as recently used
        return cache[key]
    
    def put(key, value):
        if key in cache:
            cache.move_to_end(key)
        cache[key] = value
        if len(cache) > capacity:
            cache.popitem(last=False)  # remove least recently used
    
    return get, put

**⭐ 5. Detailed Example**

Input:

Capacity = 2

Operations: put(1, 'A'), put(2, 'B'), get(1), put(3, 'C'), get(2)

In [0]:
get, put = lru_cache(2)
put(1, 'A')
put(2, 'B')
print(get(1))  # 'A'
put(3, 'C')    # evicts key 2
print(get(2))  # -1

A
-1


**⭐ 6. Mini Practice Problems**

Implement an LRU cache that tracks user session data with capacity 3.

Given a stream of log events, maintain the last 5 unique event IDs in access order.

Extend the LRU cache to return the current cache keys in MRU → LRU order.

**⭐ 7. Full Data Engineering Scenario**

Problem Statement:
You are building a real-time recommendation cache. Store the last 100 items viewed by each user to avoid recomputation of recommendations.

Expected Output:

Cache always contains most recent 100 items per user

Evict the oldest item if the user exceeds capacity

In [0]:
from collections import OrderedDict

user_cache = defaultdict(lambda: OrderedDict())
capacity = 100

def access_item(user_id, item_id):
    cache = user_cache[user_id]
    if item_id in cache:
        cache.move_to_end(item_id)
    cache[item_id] = True
    if len(cache) > capacity:
        cache.popitem(last=False)


**⭐ 8. Time & Space Complexity**

Time Complexity: O(1) for get and put (OrderedDict maintains a doubly-linked list + hash map)

Space Complexity: O(n) where n = cache capacity

**⭐ 9. Common Pitfalls & Mistakes**

❌ Using a regular dict (Python 3.7+ preserves insertion order but doesn’t support move_to_end efficiently)
❌ Forgetting to move accessed items to the end → cache order breaks
❌ Not evicting items when capacity exceeded → memory grows unbounded

✔ Always use OrderedDict for LRU eviction
✔ Use move_to_end for marking recently used
✔ Use popitem(last=False) to evict least recently used